In [1]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from ARIMA import ARIMA_forecasts, ARIMA_forecast
from random_walk import random_walk_forecast
from helper_functions import compute_rmse, load_monthly_baci_data
import numpy as np

# Prepare the dataset
def create_ann_dataset(data, lookback, is_test=False, ):
    X, y = [], []
    if is_test:  # for test data, we just need the last entry for 1-step ahead forecast
        X.append(data[-lookback:])
        return np.array(X).reshape(1, lookback, 1), None
    else:
        for i in range(lookback, len(data)):
            X.append(data[i-lookback:i])
            y.append(data[i])
        return np.array(X), np.array(y)

# Design the ANN model
def create_ann_model(lookback):
    model = Sequential()
    model.add(Dense(8, input_dim=lookback, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def ANN_forecast(train, test, steps_ahead, lookback=2):
    # Rescale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test.values.reshape(-1, 1))
    
    X_train, y_train = create_ann_dataset(train_scaled, lookback)
    
    model = create_ann_model(lookback)
    model.fit(X_train, y_train, epochs=50, batch_size=1)

     # Starting with the last 'lookback' data points from the training set
    input_data = train_scaled[-lookback:]
    
    # List to store forecasted points
    scaled_forecast = []

    # Iteratively predict steps ahead
    for _ in range(steps_ahead):
        X_test, _ = create_ann_dataset(input_data, lookback, is_test=True)
        prediction = model.predict(X_test)
        scaled_forecast.append(prediction[0][0])  # Add the new prediction
        
        # Append the new prediction to the end of input_data and remove the oldest value
        input_data = np.append(input_data[1:], prediction)
    # Inverse the scaling on the forecasted values
    forecast = scaler.inverse_transform(np.array(scaled_forecast).reshape(-1, 1))
    
    return forecast.flatten()

def ARIMA_vs_RW_vs_ANN(data, train_ratio=0.8, months_ahead=1, n_forecasts=1):
    train_size = int(train_ratio * len(data))
    arima_rmses, rw_rmses, ann_rmses = [], [], []

    for _ in range(n_forecasts):
        train = data["CLOSE"].iloc[:train_size]
        test = data["CLOSE"].iloc[train_size:train_size + months_ahead]

        # ARIMA forecast
        arima_forecast = ARIMA_forecast(train, test, months_ahead)
        arima_rmse = compute_rmse(test, arima_forecast)
        arima_rmses.append(arima_rmse)

        # Random Walk forecast
        rw_forecast = random_walk_forecast(train, test)
        rw_rmse = compute_rmse(test, rw_forecast)
        rw_rmses.append(rw_rmse)

        # ANN forecast
        ann_forecast = ANN_forecast(train=train, test=test, steps_ahead=months_ahead)
        ann_rmse = compute_rmse(test, ann_forecast)
        ann_rmses.append(ann_rmse)

        train_size += months_ahead

    return arima_rmses, rw_rmses, ann_rmses

df = load_monthly_baci_data()
arima_rmses, rw_rmses, ann_rmses = ARIMA_vs_RW_vs_ANN(df, n_forecasts=30, months_ahead=2)
print(f"ARIMA RMSEs: {np.average(arima_rmses)}")
print(f"Random Walk RMSEs: {np.average(rw_rmses)}")
print(f"ANN RMSEs: {np.average(ann_rmses)}")


/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


Epoch 1/50
  1/234 [..............................] - ETA: 27s - loss: 0.0309

/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariusbolstad/VSCode/project-thesis/random_walk.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return [train[-1]] * len(test)


234/234 [==============================] - 0s 999us/step - loss: 0.0662
Epoch 2/50
234/234 [==============================] - 0s 428us/step - loss: 0.0302
Epoch 3/50
234/234 [==============================] - 0s 392us/step - loss: 0.0199
Epoch 4/50
234/234 [==============================] - 0s 392us/step - loss: 0.0142
Epoch 5/50
234/234 [==============================] - 0s 392us/step - loss: 0.0110
Epoch 6/50
234/234 [==============================] - 0s 517us/step - loss: 0.0091
Epoch 7/50
234/234 [==============================] - 0s 380us/step - loss: 0.0083
Epoch 8/50
234/234 [==============================] - 0s 382us/step - loss: 0.0081
Epoch 9/50
234/234 [==============================] - 0s 396us/step - loss: 0.0077
Epoch 10/50
234/234 [==============================] - 0s 515us/step - loss: 0.0074
Epoch 11/50
234/234 [==============================] - 0s 385us/step - loss: 0.0072
Epoch 12/50
234/234 [==============================] - 0s 485us/step - loss: 0.0069
Epoch 13/50


/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariusbolstad/VSCode/project-thes

...

288/288 [==============================] - 0s 598us/step - loss: 0.0276
Epoch 2/50
288/288 [==============================] - 0s 395us/step - loss: 0.0114
Epoch 3/50
288/288 [==============================] - 0s 503us/step - loss: 0.0076
Epoch 4/50
288/288 [==============================] - 0s 396us/step - loss: 0.0061
Epoch 5/50
288/288 [==============================] - 0s 388us/step - loss: 0.0056
Epoch 6/50
288/288 [==============================] - 0s 391us/step - loss: 0.0054
Epoch 7/50
288/288 [==============================] - 0s 512us/step - loss: 0.0052
Epoch 8/50
288/288 [==============================] - 0s 389us/step - loss: 0.0050
Epoch 9/50
288/288 [==============================] - 0s 395us/step - loss: 0.0049
Epoch 10/50
288/288 [==============================] - 0s 398us/step - loss: 0.0047
Epoch 11/50
288/288 [==============================] - 0s 496us/step - loss: 0.0048
Epoch 12/50
288/288 [==============================] - 0s 399us/step - loss: 0.0046
Epoch 13/50


/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariusbolstad/VSCode/project-thes

290/290 [==============================] - 0s 606us/step - loss: 0.0392
Epoch 2/50
290/290 [==============================] - 0s 490us/step - loss: 0.0240
Epoch 3/50
290/290 [==============================] - 0s 398us/step - loss: 0.0226
Epoch 4/50
290/290 [==============================] - 0s 395us/step - loss: 0.0182
Epoch 5/50
290/290 [==============================] - 0s 388us/step - loss: 0.0147
Epoch 6/50
290/290 [==============================] - 0s 510us/step - loss: 0.0120
Epoch 7/50
290/290 [==============================] - 0s 402us/step - loss: 0.0102
Epoch 8/50
290/290 [==============================] - 0s 396us/step - loss: 0.0088
Epoch 9/50
290/290 [==============================] - 0s 388us/step - loss: 0.0078
Epoch 10/50
290/290 [==============================] - 0s 476us/step - loss: 0.0072
Epoch 11/50
290/290 [==============================] - 0s 391us/step - loss: 0.0066
Epoch 12/50
290/290 [==============================] - 0s 407us/step - loss: 0.0062
Epoch 13/50


/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariusbolstad/VSCode/project-thes

Epoch 1/50
292/292 [==============================] - 0s 563us/step - loss: 0.0058
Epoch 2/50
292/292 [==============================] - 0s 386us/step - loss: 0.0056
Epoch 3/50
292/292 [==============================] - 0s 391us/step - loss: 0.0055
Epoch 4/50
292/292 [==============================] - 0s 493us/step - loss: 0.0054
Epoch 5/50
292/292 [==============================] - 0s 424us/step - loss: 0.0054
Epoch 6/50
292/292 [==============================] - 0s 399us/step - loss: 0.0053
Epoch 7/50
292/292 [==============================] - 0s 395us/step - loss: 0.0052
Epoch 8/50
292/292 [==============================] - 0s 508us/step - loss: 0.0052
Epoch 9/50
292/292 [==============================] - 0s 384us/step - loss: 0.0051
Epoch 10/50
292/292 [==============================] - 0s 402us/step - loss: 0.0052
Epoch 11/50
292/292 [==============================] - 0s 403us/step - loss: 0.0051
Epoch 12/50
292/292 [==============================] - 0s 482us/step - loss: 0.0050
E